# Working with Time Zones

## Creating and working with time-zone objects

The standard way to create a datetime literal is to attach it to the constructor by passing it to the `tzinfo` argument of the constructor:

In [ ]:
from datetime import datetime, timezone, timedelta
from dateutil import tz

In [ ]:
from helper_functions import print_dt_tzinfo

In [ ]:
EASTERN = tz.gettz("America/New_York")

In [ ]:
dt = datetime(2017, 8, 11, 14, tzinfo=EASTERN)
print_dt_tzinfo(dt)

If you have a naïve wall time or a wall time in another zone that you want to translate without shifting, use `datetime.replace`:

In [ ]:
print_dt_tzinfo(dt.replace(tzinfo=tz.gettz('America/Los_Angeles')))

If you have an *absolute* time, in UTC or otherwise, and you want to represent it in another timezone, use `datetime.astimezone`:

In [ ]:
print_dt_tzinfo(dt.astimezone(tz.gettz('America/Los_Angeles')))

### `pytz`

In `pytz`, `datetime.astimezone()` still works as expected:

In [ ]:
import pytz

In [ ]:
print_dt_tzinfo(dt.astimezone(pytz.timezone('America/Los_Angeles')))

But the constructor and `.replace` methods fail horribly:

In [ ]:
print_dt_tzinfo(dt.replace(tzinfo=pytz.timezone('America/Los_Angeles')))

This is because `pytz` uses a different time zone model. `pytz`'s time zone model implements the `tzinfo` interface *statically*, which is to say that the `tzname`, `utcoffset` and `dst` are all calculated eagerly, when the `tzinfo` is attached to the `datetime`.

In order to accomplish this, `pytz` expects all `tzinfo` objects to be attached to the `datetime` *by the time zone object itself*, using the `localize()` method:

In [ ]:
EASTERN_p = pytz.timezone('America/New_York')
dt_p = EASTERN_p.localize(datetime(2017, 8, 11, 14))
print_dt_tzinfo(dt_p)

This also means that unlike with normal `tzinfo` objects, after you've done some arithmetic on a `pytz`-aware `datetime` object, you must `normalize` it:

In [ ]:
print('dateutil:')
print_dt_tzinfo(dt + timedelta(days=180))
print('')
print('pytz')
print_dt_tzinfo(dt_p + timedelta(days=180))

In [ ]:
print_dt_tzinfo(EASTERN_p.normalize(dt_p + timedelta(days=180)))

### `datetime.now` and `datetime.fromtimestamp`

The alternate constructors `now()` and `fromtimestamp` both take an optional argument `tz`:


```python
    def now(tz=None):
        """Get the datetime representing the current time"""
        # ...

    def fromtimestamp(self, timestamp, tz=None):
        """ Return the datetime corresponding to a POSIX timestamp """
        # ...
```

If the `tz` parameter is not passed, they will return a *naïve* datetime, representing the time in your system local time.

In [ ]:
datetime.fromtimestamp(1577854800.0)

If you want an *aware* timezone from a timestamp (or the current time), pass it to the `tz` parameter, and it will calculate the correct `datetime` matching that time zone. This works with both `pytz` and `dateutil.tz`:

In [ ]:
print("pytz:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN_p))
print("")
print("dateutil:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN))

**Note**: There are also the semi-deprecated `datetime.utcnow()` and `datetime.utcfromtimestamp()` functions. These  return a *naïve* datetime expressed in UTC. It is almost always better to simply pass `tz=UTC` (where `UTC` is some time zone object), which will give you an *aware* `datetime`.

### Exercise: Current time in multiple time zones


To practice using time zones, try writing a function that takes as inputs a list of IANA time zones and prints the time in each time zone.

So, for example:

```python
>>> now_in_zones(['Asia/Tokyo',
...               'Europe/Berlin',
...               'America/New_York',
...               'America/Los_Angeles'])
```
```
Asia/Tokyo:               2020-01-01 14:00:00+09:00
Europe/Berlin:            2020-01-01 06:00:00+01:00
America/New_York:         2020-01-01 00:00:00-05:00
America/Los_Angeles:      2019-12-31 21:00:00-08:00
```

In [ ]:
def now_in_zones(tz_list):
    print("Needs implementation!")

now_in_zones(['Asia/Tokyo',
              'Europe/Berlin',
              'America/New_York',
              'America/Los_Angeles'])
    